In [1]:
from multiprocessing import Pool
import numpy as np
import os
import pathlib
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Generate list of countries or states currently in the repository (ours, not JHU's)
states = [x.name.split('.')[0].split('_')[-1]
                    for x in pathlib.Path('../data').iterdir()
                    if 'covid_timeseries_US_' in str(x)]
os.makedirs('fits', exist_ok=True)

In [3]:
from __init__ import load_or_compile_stan_model
from MBS_epidemic_ccclinearrd_fullinearmodel import go

In [4]:
%time sm = load_or_compile_stan_model('MBS_epidemic_ccclinearrd_fullinearmodel', force_recompile=False)

Loading from cache...
CPU times: user 22.8 ms, sys: 62.2 ms, total: 85 ms
Wall time: 82.1 ms


In [5]:
def run(sm, state):
    max_iter=100
    roi = 'US_%s' % state
    i = 0
    while i<max_iter:
        try:
            go(sm, roi)
        except RuntimeError:
            i+=1
        else:
            return "%s succeeded on iteration %d" % (state, i)
    return "%s failed after %d iterations" % (state, max_iter)

In [ ]:
pool = Pool(processes=20)
results = [pool.apply_async(run, [sm, state]) for state in states]
answers = [result.get() for result in results]

In [ ]:
answers